In [125]:
import sys
import os
import time
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm

In [126]:
# 获取域名元数据
metad = {}
with open("/root/Workspace/DWTF/domain_relation/src/secrank1k-metadata.txt","r") as inf:
    for line in tqdm(inf.readlines(), desc="Get domains' meta data"):
        td = eval(line.strip())
        metad[td['sld']] = td

Get domains' meta data:   0%|          | 0/1000 [00:00<?, ?it/s]

In [127]:
## 有向图转json
def graph2json(G):
    d = {'nodes':[], 'links':[]}
    # 节点信息
    for n in G.nodes():
        d['nodes'].append({
            'id':n,
            'indegree': G.in_degree(n),
            'color': G.nodes[n]['color']
        })
    # 边信息
    d['links'] = [{'source': u, 'target': v, 'color':G.edges[u,v]['color']} for u, v in G.edges()]
    return d

In [128]:
## 去除自环
def wipecycle(G):
    remove_edges = []
    for u,v in G.edges():
        if u == v:
            remove_edges.append((u,v))
    for i in remove_edges:
        G.remove_edge(i[0], i[1])

In [129]:
# 给图(json)增加其他元数据
def addmetadata(j):
    for domain in tqdm(j['nodes'], desc="Add mate data to josn"):
            try:
                domain['requestCount'] = metad[domain['id']]['requestCount']
                domain['clientIpCount'] = metad[domain['id']]['clientIpCount']
                domain['subdomainCount'] = metad[domain['id']]['subdomainCount']
            except Exception as e:
                pass

In [130]:
# 增加点和线的颜色属性

## 增加默认的绿色属性
def addg(G):
    # 添加节点属性
    for i in G.nodes():
        nx.set_node_attributes(G, {i:{'color':'green'}})
    # 添加边的属性
    for e in G.edges():
        nx.set_edge_attributes(G, {e:{'color':'green'}})

def addr(G, breakN, breakE):
    # 更改节点属性
    for i in breakN:
        nx.set_node_attributes(G, {i:{'color':'red'}})
    # 更改边的属性
    for e in breakE:
        nx.set_edge_attributes(G, {e:{'color':'red'}})

In [131]:
## block一个域名，返回受影响的线和边
# TODO: 没有考虑到互换的情况，应该增加一种判断，一个点到不了根节点就算失效，但边可以先保留

def blockdomain(G, domainname, breakE, breakN):
    breakN.append(domainname)
    print("[++++++]"+domainname+" blocked !")
    # 获取所有指向该点的边，入列
    for edge in G.in_edges(domainname):
        print("[------]edge {} donw".format(edge))
        breakE.append(edge)
        
    # 获取所有指向该点的点, 判断是否需要继续BAN，深度有限
    for src in G.predecessors(domainname):
        # 判断是否所有的outedge都已不可用
        for edge in G.out_edges(src):
            if edge not in breakEdges:
                break
            #所有出边均已不可用，下线该点
            blockdomain(G, domainname=src, breakE=breakE, breakN=breakN)
    
##Test
# breakEdges, breakNods = [],[]
# blockdomain(nxs['edu']['general'], 'edu.cn', breakEdges, breakNods)
# breakEdges = list(set(breakEdges))
# breakNods = list(set(breakNods))

In [132]:
## 提取指定有向图，输出默认颜色（green）的json文件
import networkx as nx
from networkx.readwrite import json_graph
import json
from pprint import pprint

tars = ['edu', 'gov', 'other']
nxs = {}
for i in tars:
    nxs[i] = pickle.load(open("/root/Workspace/DWTF/domain_relation/data/%s.graph_set_global.bin" % i, "rb"))

wc = True

for u,v in nxs.items():
    if wc:
        wipecycle(v['general'])
    #增加默认的颜色属性
    addg(v['general'])
    j = graph2json(v['general'])
    # 添加其他元数据
    addmetadata(j)

    json_data = json.dumps(j, indent=4)
    with open("../data/%s.graph_set_global.json"%u, "w+") as ouf:
        ouf.write(json_data)

Add mate data to josn:   0%|          | 0/175 [00:00<?, ?it/s]

Add mate data to josn:   0%|          | 0/215 [00:00<?, ?it/s]

Add mate data to josn:   0%|          | 0/332 [00:00<?, ?it/s]

In [ ]:
# block掉某个域名后的情况

#首先获取完整的有向图数据,并做基本处理
tars = ['edu', 'gov', 'other']
nxs = {}
for i in tars:
    nxs[i] = pickle.load(open("/root/Workspace/DWTF/domain_relation/data/%s.graph_set_global.bin" % i, "rb"))

wc = True
for u,v in nxs.items():
    if wc:
        # 去除自环
        wipecycle(v['general'])
    #增加默认的颜色属性
    addg(v['general'])
    
# block a domain
breakEdges, breakNods = [],[]
blockdomain(nxs['edu']['general'], 'edu.cn', breakEdges, breakNods)
breakEdges = list(set(breakEdges))
breakNods = list(set(breakNods))

addr(nxs['edu']['general'], breakNods, breakEdges)

j = graph2json(nxs['edu']['general'])
# 添加其他元数据
addmetadata(j)
json_data = json.dumps(j, indent=4)
with open("../data/%s.graph_set_global.block%s.json"%("edu", "edu.cn"), "w+") as ouf:
    ouf.write(json_data)